In [2]:
from datetime import datetime
import sys
import time
import sqlite3
import pandas as pd
from gensim.parsing.preprocessing import strip_multiple_whitespaces
import gensim

In [3]:
def initialize_db(db_name):
    connection = sqlite3.connect(db_name)
    connection.row_factory = sqlite3.Row

    cur = connection.cursor()
    cur.execute("SELECT * FROM Jobs")

    return connection, cur

def parse_job(row):
    return {
        'organization' : row['organization'],
        'job_title' : row['job_title'],
        'number_of_openings' : row['number_of_openings'],
        'address_line1' : row['address_line1'],
        'city' : row['city'],
        'province_state' : row['province_state'],
        'postal_code' : row['postal_code'],
        'job_summary' : row['job_summary'],
        'job_responsibilities' : row['job_responsibilities'],
        'required_skills' : row['required_skills'],
        'targeted_disciplines' : row['targeted_disciplines'].replace('\r', '').replace('\t', ' ').replace('\n', '').replace('Targeted Clusters', '')
    }

In [4]:
connection, cur = initialize_db("data/fall2017/jobs.db")

OperationalError: unable to open database file

In [ ]:
count = 0
jobs = []
while True:
    count += 1
    row = cur.fetchone()

    if row is None:
        print("========================== All records processed ============================")
        break

    job = parse_job(row)
    
    jobs.append(job)
    
print("Number of rows loaded: ", count)
connection.close()

In [ ]:
#save to pandas dataframe
df = pd.DataFrame.from_dict(jobs, orient="columns")

In [ ]:
df.to_parquet('data/data.parquet', compression="gzip", engine='fastparquet')

In [69]:
#load
data = pd.read_parquet("data/data.parquet", engine="fastparquet")

In [70]:
data["targeted_disciplines"].values[3090]

'              - Theme - Computing: Software           - Theme - Data Analytics, Statistical Analysis and Mathematical Optimization            ENG - Software Engineering           MATH - Applied Mathematics           MATH - Computer Science           MATH - Statistics and Actuarial Science'

In [71]:
def get_cat(row):
    if ("ENG - Software Engineering" in row["targeted_disciplines"]) and ("MATH - Computer Science" in row["targeted_disciplines"]):
        return True
    else:
        return False

In [72]:
def clean(x):
    newstr = strip_multiple_whitespaces(x)
    return newstr


In [73]:
#Clean data
data["job_responsibilities"] = data["job_responsibilities"].map(lambda x: clean(x))
data["job_summary"] = data["job_summary"].map(lambda x: clean(x))
data["required_skills"] = data["required_skills"].map(lambda x: clean(x))

In [74]:
m = data.apply(get_cat, axis=1)
data = data[m]

In [75]:
data.head()

,address_line1,city,job_responsibilities,job_summary,job_title,number_of_openings,organization,postal_code,province_state,required_skills,targeted_disciplines
1,187 King Street South,Waterloo,Responsibilities -Under the direction of staff...,"Validus Research Inc., based in Waterloo, Onta...",Developer,1,Validus Group,N2J 1R1,Ontario,Essential Skills/Qualifications -Undergraduate...,- Theme - Business Administratio...
11,"144 Front Street West, Suite 685",Toronto,Peraso's Product Validation engineers are resp...,Company Profile Peraso is a fabless semiconduc...,Validation Engineering,1,Peraso Technologies Inc,M5J 2L7,Ontario,Minimum Qualifications: - Understanding of C/C...,- Theme - Computing: Information...
12,67 Yonge Street,Toronto,We are looking for a QA Automation Developer t...,"QuickTapSurvey, created by TabbleDabble helps ...",Automation Developer,1,QuickTapSurvey created by TabbleDabble Inc.,M5E 1J8,Ontario,You must have development skills that consist ...,- Theme - Computing: Information...
14,609 Kumpf Drive,Waterloo,- Build strong partnerships with software engi...,As part of the Global R&D; Software organizati...,Technical Writer,1,DEMATIC,N2V 1K8,Ontario,- Exceptional writing skills - Proficiency wit...,- Theme - Computing: Information...
15,,,RESPONSIBILITIES Review and develop understand...,Are you looking for the opportunity to be invo...,Manufacturing Software Development,1,Ford Motor Company of Canada Limited,,Ontario,ESSENTIAL SKILLS AND QUALIFICATIONS Working to...,- Theme - Computing: Hardware ...


In [76]:
#create corpus
def read_corpus():
    for i, (doc1, doc2, doc3) in enumerate(zip(data["job_responsibilities"].values, data["job_summary"].values, data["required_skills"].values)):
        doc = doc1 + " " + doc2 + " " + doc3
        yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(doc), [i])

train_corpus = list(read_corpus())

In [77]:
train_corpus[:2]

[TaggedDocument(words=['under', 'the', 'direction', 'of', 'staff', 'developers', 'develop', 'new', 'or', 'enhance', 'existing', 'vcaps', 'features', 'as', 'well', 'as', 'fix', 'production', 'issues', 'participate', 'in', 'design', 'meetings', 'code', 'reviews', 'and', 'other', 'development', 'activities', 'develop', 'in', 'javascript', 'java', 'share', 'experiment', 'prototype', 'with', 'latest', 'technologies', 'and', 'development', 'approaches', 'rest', 'angularjs', 'attentive', 'to', 'details', 'and', 'be', 'willing', 'to', 'take', 'ownership', 'for', 'assigned', 'development', 'tasks', 'validus', 'research', 'inc', 'based', 'in', 'waterloo', 'ontario', 'is', 'wholly', 'owned', 'subsidiary', 'of', 'validus', 'holdings', 'validus', 'holdings', 'ltd', 'is', 'publicly', 'traded', 'nyse', 'vr', 'bermuda', 'based', 'catastrophe', 'reinsurance', 'company', 'with', 'over', 'billion', 'usd', 'in', 'assets', 'we', 'have', 'openings', 'for', 'co', 'op', 'developers', 'to', 'join', 'the', 'vca

In [92]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=2, epochs=40)

In [93]:
model.build_vocab(train_corpus)

In [94]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

Wall time: 22.3 s


In [95]:
#resume = "Secured over $15k+ worth of sponsorship for over 6 events (1200+ attendees combined) since inception. Hosted one of Canada’s first game development hackathons with Red Bull Canada: redbull.com/adrenalan"
resume = "Machine Learning Intern Worked on proof-of-concept projects to demonstrate ML feasibility in key supply chain areas. \
Pre-processed time series data, trained forecasting models, and evaluated predictions. \
Mentored other interns on data science libraries, Jupyter, natural language processing, and ML best practices. "
# Secured over $15k+ worth of sponsorship for over 6 events (1200+ attendees combined) since inception. \
# Created fully functional platform to collect hackathon data for analytics. \
# Hosted one of Canada’s first game development hackathons with Red Bull Canada: redbull.com/adrenalan \
# Experimented with state of the art NLP techniques to detect type of toxicity in online comments. \
# Experimented with LSTMs, word embeddings (Glove, FastText), and techniques such as sequence bucketing. \
# Fine-tuned and ensembled Bidirectional Encoder Representations from Transformers (BERT). \
# Integrated an “auto-swipe” feature into Tinder based on real-time brain EEG data collected by the Muse. \
# Tested on participants at Hack The 6ix 2017 and achieved a 75% accuracy. \
# Knowledgeable in: Git, Docker, Python (Pandas, Numpy, Scikit-Learn), Keras, Pytorch, Java, Javascript, VueJS, Firebase \
# "

print(gensim.utils.simple_preprocess(resume))
inferred_vector = model.infer_vector(gensim.utils.simple_preprocess(resume))
sims = model.docvecs.most_similar([inferred_vector], topn=50)

['machine', 'learning', 'intern', 'worked', 'on', 'proof', 'of', 'concept', 'projects', 'to', 'demonstrate', 'ml', 'feasibility', 'in', 'key', 'supply', 'chain', 'areas', 'pre', 'processed', 'time', 'series', 'data', 'trained', 'forecasting', 'models', 'and', 'evaluated', 'predictions', 'mentored', 'other', 'interns', 'on', 'data', 'science', 'libraries', 'jupyter', 'natural', 'language', 'processing', 'and', 'ml', 'best', 'practices']


In [96]:
sims

[(61, 0.4614437222480774),
 (553, 0.42867788672447205),
 (805, 0.42193710803985596),
 (1173, 0.4186103343963623),
 (233, 0.4173838496208191),
 (888, 0.4109939634799957),
 (381, 0.4108842611312866),
 (931, 0.40623152256011963),
 (408, 0.4061116576194763),
 (462, 0.3995017409324646),
 (755, 0.39083775877952576),
 (546, 0.38691163063049316),
 (1144, 0.385915070772171),
 (337, 0.38471347093582153),
 (1088, 0.3809712529182434),
 (1295, 0.38018345832824707),
 (173, 0.3798932433128357),
 (1007, 0.37841668725013733),
 (474, 0.378090500831604),
 (554, 0.3769749402999878),
 (276, 0.37686070799827576),
 (200, 0.37106969952583313),
 (1031, 0.37083327770233154),
 (745, 0.36703556776046753),
 (706, 0.3646235466003418),
 (1235, 0.3631197512149811),
 (792, 0.3596944212913513),
 (615, 0.3595716953277588),
 (1036, 0.3591802418231964),
 (823, 0.3589065372943878),
 (899, 0.35719573497772217),
 (855, 0.3563346266746521),
 (652, 0.3559548258781433),
 (1039, 0.35582977533340454),
 (777, 0.3557756543159485),


In [97]:
for job in sims:
    X = job[0]
    print("ID: ", X)
    print(data["job_title"].values[X])
    print(data["job_responsibilities"].values[X])
    print(data["job_summary"].values[X])
    print(data["required_skills"].values[X])
    print("================================")

ID:  61
Artificial Intelligence Research
AI is not just a buzzword for Wish. It forms the foundational layer of everything we do. We treat data as a natural resource and the projects you work on will push the boundaries of AI at massive scale. You will get a great deal of responsibility for each project you work on, and will have the flexibility to influence design and execution. The field of AI is moving quickly and we need people who can adapt to build state of the art systems. Some of the most exciting areas you will work on are computer vision, NLP and deep reinforcement learning.
Our engineers move extremely fast, while solving unusual and challenging problems. They're smart, hands-on, and have a solid foundation in computer science, math or software engineering. They have strong competencies in data structures, algorithms and software design, and are able to contribute under a great deal of independence. They also have a strong desire and passion for hacking social commerce.
-Pas